


---


Risk analysis is crucial in order to understand the areas to prioritize interventions. Assessment of drought risk using the INFORM framework entails the combination of hazard and exposure, vulnerability and coping capacity components. Each component is estimated by combining a number of weighted indicators. 

The risk analysis was conducted at county level and data for the different indicators were sourced from KNBS and FEWSNET.

RISK = Hazard & Exposure * Vulnerability * Lack of Coping Capacity






**Hazard & Exposure** = SPI



**Vulnerability Index** = (Poverty incidence * 0.18) + (Gini Index * 0.15) + (Literacy rates   * 0.19) + (Access to Improved water sources   * 0.15) + (Access to Improved Sanitation * 0.14) + (Integrated Food Security Phase Classification * 0.19) 

**Lack of Coping capacity** = (Access to Improved water sources   * 0.33) + (Travel Time to the Nearest City * 0.24) + (Unemployment * 0.38)


The input of this notebook is indicator data in form of a csv and its output the normalized indicators and risk components in csv format.




In [3]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

from google.colab import files

Import Indicator csv file

In [5]:
uploaded = files.upload()
myIndicators = next(iter(uploaded))

Saving indicators.csv to indicators (1).csv


In [6]:
# read csv
mydf = pd.read_csv(myIndicators)

In [9]:
#show column names in the csv

print (mydf.columns)

Index(['County ', 'Literacy rate %', 'Poverty severity rate %',
       'Access drinking water', 'Improved sanitation', 'Gini Coefficient %',
       'FIP Frequency', 'Travel times', 'water sources', 'Unemployment %',
       'SPI Frequency'],
      dtype='object')


In [10]:
# Minmax normalization functions

def normalize_column (c):

    nc = (c- c.min()) /(c.max()- c.min())

    return nc


def inv_normalize_column (c):

    nc = (c- c.min()) /(c.max()- c.min())

    nc = 1-nc

    return nc

In [12]:
# specify which columns are to be normalized

inverse_columns = ['Literacy rate %','Access drinking water','Improved sanitation', 'water sources']

standard_columns = ['Poverty severity rate %','Gini Coefficient %','Travel times','Unemployment %','SPI Frequency','FIP Frequency']


In [13]:
#perform normalization

mydf[standard_columns]= normalize_column (mydf[standard_columns])

mydf[inverse_columns]=inv_normalize_column (mydf[inverse_columns])

In [14]:
# export normalized csv

mydf.to_csv('normalized_indicators.csv', index= False)
files.download('normalized_indicators.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# combine weighted components

mydf['hazard_component'] = mydf['SPI Frequency']

mydf['vulnerability_component']  = (mydf['Literacy rate %'] * 0.19) + (mydf['Poverty severity rate %'] * 0.18)\
                          + (mydf['Access drinking water'] * 0.15 ) + (mydf['Gini Coefficient %'] *  0.15) \
                          + (mydf['Improved sanitation'] *0.14 ) + (mydf['FIP Frequency'] * 0.19 )


mydf['coping_component'] = (mydf['Travel times'] * 0.285 ) + (mydf['water sources'] * 0.333 )\
                   + (mydf['Unemployment %'] * 0.382 ) 



In [16]:
# normalize the drought risk components

components = ['hazard_component', 'vulnerability_component', 'coping_component']

mydf[components]= normalize_column (mydf[components])

In [17]:
# calculate drought risk

mydf['drought_risk'] =  mydf['hazard_component'] * mydf['vulnerability_component'] * mydf['coping_component']


# normalize drought risk
mydf['drought_risk_normalized'] = normalize_column(mydf['drought_risk'])

In [18]:
#export the three components

mydf.to_csv('components.csv',\
            columns = ['County ','drought_risk_normalized','drought_risk','hazard_component','vulnerability_component','coping_component'], index= False)

files.download('components.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>